In [1]:
!pip install pyod tensorflow keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.4 MB/s eta 0:00:0000:0100:01
  Created wheel for pyod: filename=pyod-2.0.2-py3-none-any.whl size=198478 sha256=842f1aed5a90248d7e906e7aed62ea14cec9d688ca0ec3e4a5d1801dc172f84b
  Stored in directory: /root/.cache/pip/wheels/77/c2/20/34d1f15b41b701ba69f42a32304825810d680754d509f91391
Successfully built pyod
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np
import pandas as pd
from pyod.models.knn import KNN
from matplotlib import pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, RepeatVector, TimeDistributed
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-09-23 12:27:22.948552: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 12:27:22.948653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 12:27:23.072889: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/dataset1/Dataset 1 .csv
/kaggle/input/dataset-1/Dataset_1.csv
/kaggle/input/dataset2/Dataset 2 .csv


In [3]:
# Load the data
df = pd.read_csv("/kaggle/input/dataset2/Dataset 2 .csv", nrows=50000)

# Convert 'Datetime' column to datetime format
df['Datetime'] = pd.to_datetime(df['Date\t'] + ' ' + df['Time'], format='%d-%m-%Y %M:%S.%f')
df.set_index('Datetime', inplace=True)
df.drop(['Date\t', 'Time'], axis=1, inplace=True)

cumulative_hour_diff = 0
prev_minute = df.index[0].minute
prev_date = df.index[0].date()

# Adjust for non-uniform sampling
def adjust_hour(index):
    global cumulative_hour_diff, prev_minute, prev_date
    if index.date() != prev_date:
        cumulative_hour_diff = 0  # Reset hour difference at the start of a new day
        prev_date = index.date()
    elif index.minute < prev_minute:
        cumulative_hour_diff += 1
    prev_minute = index.minute
    return index + pd.Timedelta(hours=cumulative_hour_diff)

df.index = df.index.map(adjust_hour)
df = df.iloc[:50000, :]
def resample_and_interpolate(group):
    avg_time_diff = group.index.to_series().diff().mean()
    return group.resample(avg_time_diff).mean().interpolate(method='time')

df = df.groupby(df.index.date).apply(resample_and_interpolate)
df.reset_index(level=0, drop=True, inplace=True)
df

,P1,P2,P3,P4,P5,P6,P7,P8,P9
Datetime,,,,,,,,,
2011-10-15 00:39:53.093501721,-0.0469,-0.0385,0.005420,0.011201,38.0,52.000000,0.0,0.0,0.0
2011-10-15 00:40:13.547292334,-0.0469,-0.0385,0.005400,0.011177,38.0,52.000000,0.0,0.0,0.0
2011-10-15 00:40:34.001082947,-0.0469,-0.0385,0.005387,0.011188,38.0,52.000000,0.0,0.0,0.0
2011-10-15 00:40:54.454873560,-0.0469,-0.0385,0.005420,0.011160,38.0,52.300000,0.0,0.0,0.0
2011-10-15 00:41:14.908664173,-0.0469,-0.0385,0.005371,0.011160,38.0,52.454545,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2011-10-25 03:45:38.857581759,-0.0469,-0.0385,0.005820,0.011310,37.0,51.777778,0.0,0.0,0.0
2011-10-25 03:45:52.715163726,-0.0469,-0.0385,0.005820,0.011310,37.0,51.833333,0.0,0.0,0.0
2011-10-25 03:46:06.572745693,-0.0469,-0.0385,0.005820,0.011310,37.0,51.888889,0.0,0.0,0.0


In [4]:
# Advanced interpolation using Gaussian Process Regression
def interpolate_with_gpr_batch(data, batch_size=25000):
    gp_data = []
    num_batches = int(np.ceil(len(data) / batch_size))
    kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))
    for col in data.columns:
        y_pred_total = []
        for batch_num in range(num_batches):
            print(batch_num+1)
            start_idx = batch_num * batch_size
            end_idx = min(start_idx + batch_size, len(data))
            batch_data = data.iloc[start_idx:end_idx]
            
            X = np.array(batch_data.index.astype(int)).reshape(-1, 1)
            y = batch_data[col].values

            # Initialize and fit Gaussian Process Regressor
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-2)
            gp.fit(X, y)

            # Predict
            y_pred, sigma = gp.predict(X, return_std=True)
            y_pred_total.extend(y_pred)

        gp_data.append(y_pred_total)

    return np.array(gp_data).T

# Run the interpolation process
df_interpolated = interpolate_with_gpr_batch(df, batch_size=10000)
df[:] = df_interpolated
df

,P1,P2,P3,P4,P5,P6,P7,P8,P9
Datetime,,,,,,,,,
2011-10-15 00:39:53.093501721,-0.0469,-0.0385,0.005420,0.011201,38.0,52.000000,0.0,0.0,0.0
2011-10-15 00:40:13.547292334,-0.0469,-0.0385,0.005400,0.011177,38.0,52.000000,0.0,0.0,0.0
2011-10-15 00:40:34.001082947,-0.0469,-0.0385,0.005387,0.011188,38.0,52.000000,0.0,0.0,0.0
2011-10-15 00:40:54.454873560,-0.0469,-0.0385,0.005420,0.011160,38.0,52.300000,0.0,0.0,0.0
2011-10-15 00:41:14.908664173,-0.0469,-0.0385,0.005371,0.011160,38.0,52.454545,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2011-10-25 03:45:38.857581759,-0.0469,-0.0385,0.005820,0.011310,37.0,51.777778,0.0,0.0,0.0
2011-10-25 03:45:52.715163726,-0.0469,-0.0385,0.005820,0.011310,37.0,51.833333,0.0,0.0,0.0
2011-10-25 03:46:06.572745693,-0.0469,-0.0385,0.005820,0.011310,37.0,51.888889,0.0,0.0,0.0


In [5]:
# Step 2: Normalize the data
scaler = RobustScaler()
df_scaled = scaler.fit_transform(df)

In [6]:
# Step 3: Initial Anomaly Detection using KNN
knn = KNN(contamination=0.03)
df['anomaly'] = knn.fit_predict(df_scaled)
df['anomaly'] = df['anomaly'].apply(lambda x: 1 if x == 1 else 0)

In [7]:
# Step 4: Identify Longest Span Without Anomalies
clean_spans = []
current_span = []

for i in range(len(df)):
    if df['anomaly'].iloc[i] == 0:
        current_span.append(df.iloc[i])
    else:
        if len(current_span) > 0:
            clean_spans.append(pd.DataFrame(current_span))
            current_span = []

if len(current_span) > 0:
    clean_spans.append(pd.DataFrame(current_span))

longest_span = max(clean_spans, key=len).drop(columns=['anomaly'])
longest_span_scaled = scaler.transform(longest_span)
longest_span_scaled.shape

(3853, 9)

In [8]:
# Step 5: Create Sliding Windows for Clean Data
def create_sliding_window(data, window_size):
    X = []
    for i in range(len(data) - window_size + 1):
        X.append(data[i:i + window_size])
    return np.array(X)

window_size = 20
X_clean = create_sliding_window(longest_span_scaled, window_size)

In [9]:
# Step 6: LSTM Autoencoder
input_dim = longest_span_scaled.shape[1]

input_layer = Input(shape=(window_size, input_dim))
lstm_layer = LSTM(90)(input_layer)
repeat_vector = RepeatVector(window_size)(lstm_layer)
lstm_output = LSTM(90, return_sequences=True)(repeat_vector)
output_layer = TimeDistributed(Dense(input_dim))(lstm_output)

autoencoder_clean = Model(inputs=input_layer, outputs=output_layer)
autoencoder_clean.compile(optimizer='adam', loss='mse')

X_clean_train = X_clean.reshape((X_clean.shape[0], window_size, input_dim))
history_clean = autoencoder_clean.fit(X_clean_train, X_clean_train, epochs=50, batch_size=32, validation_split=0.2, shuffle=False)

Epoch 1/50


I0000 00:00:1727094467.537803     122 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


96/96 [==============================] - 6s 15ms/step - loss: 0.1014 - val_loss: 0.0829
Epoch 2/50
96/96 [==============================] - 1s 7ms/step - loss: 0.0505 - val_loss: 0.0496
Epoch 3/50
96/96 [==============================] - 1s 7ms/step - loss: 0.0417 - val_loss: 0.0537
Epoch 4/50
96/96 [==============================] - 1s 7ms/step - loss: 0.0423 - val_loss: 0.0545
Epoch 5/50
96/96 [==============================] - 1s 7ms/step - loss: 0.0391 - val_loss: 0.0295
Epoch 6/50
96/96 [==============================] - 1s 7ms/step - loss: 0.0276 - val_loss: 0.0233
Epoch 7/50
96/96 [==============================] - 1s 7ms/step - loss: 0.0226 - val_loss: 0.0301
Epoch 8/50
96/96 [==============================] - 1s 7ms/step - loss: 0.0227 - val_loss: 0.0237
Epoch 9/50
96/96 [==============================] - 1s 7ms/step - loss: 0.0263 - val_loss: 0.0173
Epoch 10/50
96/96 [==============================] - 1s 7ms/step - loss: 0.0203 - val_loss: 0.0297
Epoch 11/50
96/96 [==========

In [10]:
class PolynomialRegression:
    def __init__(self, degree=2):
        self.degree = degree
        self.model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
        self.X_train = []
        self.y_train = []

    def fit(self, X, y):
        # Ensure that the model only keeps 15 values in the sliding window
        self.X_train = X[-15:]
        self.y_train = y[-15:]
        self.model.fit(self.X_train, self.y_train)

    def predict_anomaly_time(self, threshold_value, current_time):
        try:
            # Predict the time until the threshold value is reached
            X_predict = np.array([[threshold_value]])  # Predict based on the threshold value
            predicted_time = self.model.predict(X_predict)[0][0]
            return predicted_time
        except Exception as e:
            print(e)
            return np.inf  # Return infinity in case of prediction error

In [11]:
# Step 7: Validate and Predict Anomalies on Full Data using Regression for Rate of Approach
# Compute the range and normal limits
upper_limit = np.max(longest_span_scaled, axis=0)
lower_limit = np.min(longest_span_scaled, axis=0)
range_value = upper_limit - lower_limit
normal_upper_limit = upper_limit + 0.5 * range_value
normal_lower_limit = lower_limit - 0.5 * range_value
anomaly_threshold = 1.5 * range_value

In [ ]:
predictions = []
predicted_timestamps = []
regression_models = [PolynomialRegression(degree=2) for _ in range(input_dim)]

# Loop through each date and group
for date, group in df.groupby(df.index.date):
    last_index = group.index[-1]
    group_scaled = scaler.fit_transform(group)
    time_diffs = group.index.to_series().diff().dt.total_seconds().fillna(0)
    initial_window = group_scaled[:20, :-1].reshape((1, 20, input_dim))
    print(f"Checking from {group.index[0]} to {last_index}\n")
    
    # Initialize for tracking anomalies and reset conditions
    last_anomaly_timestamp = None
    normal_period_duration = 0
    # Append initial predictions from the window
    for i in range(20):
        predictions.append(initial_window[0, i, :])
    
    # Main anomaly prediction loop
    for i in range(20, len(group)):
        window = np.array(predictions[-20:]).reshape((1, 20, input_dim))
        next_value = autoencoder_clean.predict(window, verbose=0)
        predictions.append(next_value[0, -1, :])
        current_time = group.index[i]

        anomaly_detected = False
        for feature_idx in range(input_dim):
            current_value = group_scaled[i, feature_idx]
            n_upper_limit = normal_upper_limit[feature_idx]
            n_lower_limit = normal_lower_limit[feature_idx]

            # Check for anomaly
            if (current_value > n_upper_limit) or (current_value < n_lower_limit):
                anomaly_detected = True
                normal_period_duration = 0  # Reset consecutive normal count for this feature

                # Train regression on time diffs and corresponding values for this feature
                X_train = np.array(time_diffs[i-15:i].cumsum()).reshape(-1, 1)
                y_train = group.iloc[i-15:i, feature_idx].values.reshape(-1, 1)

                # Fit the polynomial regression model
                regression_models[feature_idx].fit(X_train, y_train)

                # Predict time to anomaly
                target = (n_upper_limit + anomaly_threshold[feature_idx] 
                          if current_value > n_upper_limit 
                          else n_lower_limit - anomaly_threshold[feature_idx])
                
                time_to_anomaly = regression_models[feature_idx].predict_anomaly_time(target, current_value)

                # If valid time to anomaly is predicted
                if np.isfinite(time_to_anomaly) and 0 < time_to_anomaly < 10**5:
                    predicted_timestamp = current_time + pd.Timedelta(seconds=time_to_anomaly)
                    
                    # Update or append the anomaly timestamp for this feature
                    if last_anomaly_timestamp is None or (predicted_timestamp - last_anomaly_timestamp).total_seconds() >= 300:
                        predicted_timestamps.append(predicted_timestamp)
                        last_anomaly_timestamp = predicted_timestamp
                        print(f"\tAnomaly detected at {predicted_timestamp}\n")
                    elif predicted_timestamp < predicted_timestamps[-1]:
                        predicted_timestamps[-1] = predicted_timestamp
                        print(f"\tUpdated anomaly prediction to {predicted_timestamp}\n")
                    else:
                        last_anomaly_timestamp = predicted_timestamp
        # If no anomaly detected, reset regression models
        if not anomaly_detected:
            normal_period_duration += time_diffs[i]
                
            if normal_period_duration >= 300:
                regression_models = [PolynomialRegression(degree=2) for _ in range(input_dim)]

        # Stop if current time has passed the last index for the day
        if current_time >= last_index:
            continue

Checking from 2011-10-15 00:39:53.093501721 to 2011-10-15 06:57:35.893500925

Checking from 2011-10-16 00:01:42.658140718 to 2011-10-16 23:59:54.158138534

	Anomaly detected at 2011-10-16 01:59:31.410147715

	Anomaly detected at 2011-10-16 03:34:50.935069823

	Anomaly detected at 2011-10-16 05:05:46.481966212

	Anomaly detected at 2011-10-16 15:16:20.772524627

	Anomaly detected at 2011-10-16 16:46:32.322963973

	Anomaly detected at 2011-10-16 19:48:38.083724583

Checking from 2011-10-17 00:04:06.231204500 to 2011-10-17 22:10:51.331200500

	Anomaly detected at 2011-10-17 04:01:09.709990388

	Anomaly detected at 2011-10-17 07:13:33.604854904

	Anomaly detected at 2011-10-17 12:24:00.274635269

Checking from 2011-10-18 00:08:22.262821658 to 2011-10-18 13:03:29.662818586

	Anomaly detected at 2011-10-18 06:48:02.646715073

	Anomaly detected at 2011-10-18 09:55:24.789924113

	Anomaly detected at 2011-10-18 12:03:39.029132327

	Anomaly detected at 2011-10-18 12:19:51.489273242

	Anomaly det

In [ ]:
len(predicted_timestamps)

In [ ]:
# Step 8: Visualize Results
predictions = pd.DataFrame(predictions)
num_features = df_scaled.shape[1]
fig, axes = plt.subplots(num_features, 1, figsize=(15, 5 * num_features), sharex=True)

for i in range(num_features):
    axes[i].plot(df.index, df_scaled[:, i], label=f'Feature {i+1}')
    axes[i].plot(df.index, predictions[:][i], label=f'Prediction {i+1}')
    axes[i].scatter(df.index[df['anomaly'] == 1], df_scaled[df['anomaly'] == 1, i], color='red', label='Anomalies')
    for timestamp in predicted_timestamps:
        axes[i].axvline(x=timestamp, color='blue', linestyle='--')
    axes[i].set_title(f'Feature {i+1}')
    axes[i].legend()

plt.xlabel('Datetime')
plt.show()